In [1]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats
from konlpy.tag import Okt
okt = Okt()
from mecab import MeCab
mecab = MeCab()
import pyLDAvis
import pyLDAvis.lda_model
     
client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_attraction = db['attraction_place_2023']
collection_tour_train = db['data_tour_NLP_train']
collection_tour = db['tour_contents']
data_tour = collection_tour.find()
data_tour_train = collection_tour_train.find()
df_tour = pd.DataFrame(data_tour)
df_tour_train = pd.DataFrame(data_tour_train)
df_tour

,_id,review_text,지역
0,65ee5e2b901db79521bc8779,[이벤트] 동트는 강원 취향저격 이벤트! 나에게 딱 맞는 강원 네이처 로드 코스는?,강원
1,65ee5e2b901db79521bc877a,봄 오는 소리 귀 기울이고 겨울 가는 아쉬움 녹이고,강원
2,65ee5e2c901db79521bc877b,하얀 이불 벗고 봄으로 가는 길 태평성대 가져온 봉황 눈앞에,강원
3,65ee5e2c901db79521bc877c,"열흘에 한 번, 고원 마을에 열리는 반짝 세일! 태백 통리장",강원
4,65ee5e2c901db79521bc877d,걸음 걸음 쌓이는 설렘 바다로 가는 오름길,강원
...,...,...,...
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주
49658,65ee9faac27eda916fc759b5,[윈터 바캉스] 10만원으로 떠나는 제주 당일치기,제주
49659,65ee9faac27eda916fc759b6,제주 서귀포층 패류 화석산지,제주
49660,65ee9faac27eda916fc759b7,제주 한림 용암동굴지대,제주


In [2]:
from konlpy.tag import Okt
okt = Okt()
stopwords = ['남해','대전','대전','공주','안동','나주','바퀴','보령','고령','신안','특산','공항','곡성','신상','사람','위치','가지','진도','장동','정리','도미','고창','모음','장성','시작','남원','가격','화순','국립','베스트','오토','레이스','후기','피아','중앙','공모전','태안','성주','봉화','남도','목포','군산','구례','순천','문경','완주','청주','청도','여수','울주군','포구','두기','거리','강진','뉴스','방법','통영','부여','한밭수목원','거제','서쪽','동쪽','양산','하동','포항','진주','우도','서귀포','당신', '대관령','속초','특별자치도', '삼척','영월','원주','화천','고성','양양','춘천','양구','철원','홍천','태백','달성군','구석구석','블로그','여행','여행지','경기','여주','서구','현황', '장생포','양평','가창','남평문씨','파주','영상','소개','지역','명소','요즘','마을','달성','한눈','카드','추천','수성','안산','기회','횡성','문진','발생','코스','현풍','문진','만끽']
def Tokenizer(raw, pos=["Noun"], stopword=stopwords):
    word_list = []
    for word, tag in okt.pos(raw,norm=True,stem=True):
        if len(word) > 1 and tag in pos and word not in stopword:
            if mecab.pos(word)[0][1] in ["NNG"]:
                if word == '화관':
                    word = '문화관'
                word_list.append(word)
    return " ".join(word_list)

## 데이터 분류하기

In [3]:
df_tour_train

,_id,concept,season,review_text,지역,review_text_clean
0,65ee6316a64dee66ed301e2a,0,0,[블로그 울산 남구 아이스팩 전용 수거함 어디에 있을까?,울산,아이스팩 전용 수거
1,65ee670ba64dee66ed303175,5,3,[가을여행] 전주 당일치기 여행 BEST10 (feat. 덕진공원팸투어) /가을 단...,전북,가을 전주 당일치기 진공 투어 가을 단풍
2,65ee6a64c528b55b0ae79e58,4,0,"대전 유성구 관평동 맛집, 감동적인 맛과 가격 '매봉 쌀국수와 소바'",대전,맛집 감동 쌀국수
3,65ee60fc901db79521bc991b,3,0,역사를 따라가는 DMZ 로드,경기,역사 로드
4,65ee664fc528b55b0ae78d7a,5,0,"아브라다카브라라는 주문을 외워보고 싶은, '대전 보라매공원크리스마스 풍경'",대전,주문 보라매 공원 풍경
...,...,...,...,...,...,...
9995,65ee9b78c27eda916fc74ead,5,-,서귀포 추천 여행지 신창풍차해안도로,제주,풍차 해안 도로
9996,65ee66b6901db79521bcb1d6,4,-,"반려견과 함께 하는 커피 한 잔, 광주 애견동반카페 모음.ZIP",광주,반려견 커피 애견 동반
9997,65ee6722a64dee66ed3032ef,1,-,전북 문화공간 - 완주 복합문화지구 누에(Nu-e),전북,문화 공간 복합 문화 지구 누에
9998,65ee6cd5c528b55b0ae7a7b4,5,-,"벚꽃은 갔지만 유채꽃 만발, 서울에서 만나는 남해",서울,벚꽃 유채꽃 만발


In [4]:
df_tour_train.isnull().sum()

_id                    0
concept                0
season                 0
review_text            0
지역                     0
review_text_clean    225
dtype: int64

In [5]:
df_tour_train=df_tour_train.dropna()

In [6]:
df_tour_train.isnull().sum()

_id                  0
concept              0
season               0
review_text          0
지역                   0
review_text_clean    0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
df_tour_train_train,df_tour_train_test = train_test_split(df_tour_train,test_size=0.3,random_state=42)
df_tour_train['concept'].unique()

array([0, 5, 4, 3, 1, 2, 6])

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(tokenizer = Tokenizer, use_idf=True,max_df=0.95,min_df=2,ngram_range=(1,2))
tfidfVectorizer.fit(df_tour_train['review_text_clean'])

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 2),
                tokenizer=<function Tokenizer at 0x7f0ba0d740e0>)

In [9]:
train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
concept_train_target = df_tour_train_train['concept'] # test
train_feature_test = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
concept_train_target_test = df_tour_train_test['concept']


### concept 분류 모델 만들기
- 데이터가 2000개 였을 때 : 0.6003401360544217
- 데이터가 4000개 였을 때 : 0.6909323116219668
- 데이터가 10000개 였을 때 : 0.7494033412887828

In [10]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier=RandomForestClassifier(n_jobs=-1)
randomForestClassifier.fit(train_feature,concept_train_target)

RandomForestClassifier(n_jobs=-1)

In [11]:
concept_train_feature_predict = randomForestClassifier.predict(train_feature)
concept_train_feature_predict

array([1, 5, 2, ..., 0, 1, 1])

In [21]:
from sklearn.metrics import classification_report
print(classification_report(concept_train_target,concept_train_feature_predict))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1945
           1       1.00      1.00      1.00      1656
           2       1.00      0.99      1.00       195
           3       1.00      0.99      0.99       325
           4       1.00      1.00      1.00       593
           5       1.00      1.00      1.00      1832
           6       0.99      0.98      0.99       296

    accuracy                           0.99      6842
   macro avg       1.00      0.99      0.99      6842
weighted avg       0.99      0.99      0.99      6842



In [13]:
concept_train_feature_test_predict = randomForestClassifier.predict(train_feature_test)
concept_train_feature_test_predict

array([5, 5, 0, ..., 1, 0, 1])

In [22]:
from sklearn.metrics import classification_report
print(classification_report(concept_train_target_test, concept_train_feature_test_predict))

              precision    recall  f1-score   support

           0       0.68      0.85      0.76       833
           1       0.81      0.71      0.75       729
           2       0.94      0.40      0.56        73
           3       0.84      0.44      0.58       138
           4       0.87      0.72      0.79       232
           5       0.76      0.82      0.79       793
           6       0.67      0.51      0.58       135

    accuracy                           0.75      2933
   macro avg       0.79      0.63      0.69      2933
weighted avg       0.76      0.75      0.75      2933



### season 분류 모델 만들기
- 데이터가 2000개 였을 때 : 0.95
- 데이터가 4000개 였을 때 : 0.96

In [15]:
df_tour_train.query("season != '-'")['season'].unique()

array([0, 3, 4, 1, 2], dtype=object)

In [16]:
df_tour_train_train.query("season != '-'")['review_text_clean']

655           캠핑 구산 해수욕장
178         엄마 학교 신입생 모집
3205      봉유 돼지 짬뽕 전복 짬뽕
800             기념물 이팝나무
92                항공샷 랜선
              ...       
2481         마다 평온 고이 시간
782      벚꽃 폴리스 공원 봄꽃 산책
1723         대비 면역 제철 음식
477     혈관 온라인 건강 캠페인 음식
878       외로움 간직 박물관 비양도
Name: review_text_clean, Length: 2727, dtype: object

In [17]:
season_train_feature = tfidfVectorizer.transform(df_tour_train_train.query("season != '-'")['review_text_clean']) # train
season_train_target = df_tour_train_train.query("season != '-'")['season'].astype('str') # test
season_train_feature_test = tfidfVectorizer.transform(df_tour_train_test.query("season != '-'")['review_text_clean']) # train
season_train_target_test = df_tour_train_test.query("season != '-'")['season'].astype('str') # test

In [18]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier=RandomForestClassifier(n_jobs=-1)
randomForestClassifier.fit(season_train_feature,season_train_target)

RandomForestClassifier(n_jobs=-1)

In [19]:
season_train_feature_predict = randomForestClassifier.predict(season_train_feature)
accuracy_score(season_train_target, season_train_feature_predict)

0.9996332966629996

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(season_train_target, season_train_feature_predict))

In [20]:
season_test_feature_predict = randomForestClassifier.predict(season_train_feature_test)
accuracy_score(season_train_target_test, season_test_feature_predict)

0.9688289806234204

### concept 모델 향상시키기